## Setup

In [2]:
%pip install 'vanna[chromadb,openai,postgres]'

Note: you may need to restart the kernel to use updated packages.


In [4]:
from vanna.openai import OpenAI_Chat
from vanna.base import VannaBase
from vanna.chromadb import ChromaDB_VectorStore

In [5]:



import os
from openai import OpenAI

class DeepseekLLM(VannaBase):
    def __init__(self, config=None):
        if config is None:
            raise ValueError(
                "对于 Deepseek，必须提供包含 api_key 和 model 的配置"
            )

        if "api_key" not in config:
            raise ValueError("配置必须包含 Deepseek api_key")

        if "model" not in config:
            raise ValueError("配置必须包含 Deepseek model")

        if "base_url" not in config:
            raise ValueError("配置必须包含 Deepseek base_url")

        api_key = config["api_key"]
        self.model = config["model"]
        base_url = config["base_url"]
        self.client = OpenAI(api_key=api_key, base_url=base_url)

    def system_message(self, message: str) -> any:
        return {"role": "system", "content": message}

    def user_message(self, message: str) -> any:
        return {"role": "user", "content": message}

    def assistant_message(self, message: str) -> any:
        return {"role": "assistant", "content": message}

    def generate_sql(self, question: str, **kwargs) -> str:
        # 使用父类的 generate_sql
        sql = super().generate_sql(question, **kwargs)

        # 替换 "\_" 为 "_"
        sql = sql.replace("\\_", "_")

        return sql

    def submit_prompt(self, prompt, **kwargs) -> str:
        chat_response = self.client.chat.completions.create(
            model=self.model,
            messages=prompt,
        )

        return chat_response.choices[0].message.content

class MyVanna(ChromaDB_VectorStore, DeepseekLLM):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        DeepseekLLM.__init__(self, config=config)

vn = MyVanna(config={
    "api_key": "sk-",
    "model": "deepseek-v3-baidu",
    "base_url": "https://api.302.ai/v1"
})

In [6]:
vn.connect_to_postgres(host='localhost', dbname='postgres', user='postgres', password='123456', port='5432')

## Training
You only need to train once. Do not train again unless you want to add more training data.

In [7]:

# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
# vn.train(plan=plan)



Train on Information Schema: postgres.pg_catalog pg_type
Train on Information Schema: postgres.pg_catalog pg_class
Train on Information Schema: postgres.pg_catalog pg_stat_ssl
Train on Information Schema: postgres.pg_catalog pg_index
Train on Information Schema: postgres.pg_catalog pg_stat_progress_create_index
Train on Information Schema: postgres.pg_catalog pg_attribute
Train on Information Schema: postgres.pg_catalog pg_shseclabel
Train on Information Schema: postgres.pg_catalog pg_collation
Train on Information Schema: postgres.pg_catalog pg_indexes
Train on Information Schema: postgres.pg_catalog pg_database
Train on Information Schema: postgres.pg_catalog pg_amproc
Train on Information Schema: postgres.pg_catalog pg_stat_database
Train on Information Schema: postgres.pg_catalog pg_stat_io
Train on Information Schema: postgres.pg_catalog pg_default_acl
Train on Information Schema: postgres.pg_catalog pg_stat_user_tables
Train on Information Schema: postgres.pg_catalog pg_largeobje

In [8]:

# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
CREATE TABLE IF NOT EXISTS public.new_fact_order_detail
(
    order_no character varying(255) COLLATE pg_catalog."default",
    order_time timestamp without time zone,
    order_date date,
    brand_code character varying(255) COLLATE pg_catalog."default",
    program_code character varying(255) COLLATE pg_catalog."default",
    order_type integer,
    sales numeric(18,2),
    item_qty integer,
    item_price numeric(18,2),
    channel character varying(255) COLLATE pg_catalog."default",
    subchannel character varying(255) COLLATE pg_catalog."default",
    sub_subchannel character varying(255) COLLATE pg_catalog."default",
    material_code character varying(255) COLLATE pg_catalog."default",
    material_name_cn character varying(255) COLLATE pg_catalog."default",
    material_type character varying(255) COLLATE pg_catalog."default",
    merged_c_code character varying(255) COLLATE pg_catalog."default",
    tier_code character varying(255) COLLATE pg_catalog."default",
    first_order_date date,
    is_mtd_active_member_flag integer,
    ytd_active_arr character varying(255) COLLATE pg_catalog."default",
    r12_active_arr character varying(255) COLLATE pg_catalog."default",
    manager_counter_code character varying(255) COLLATE pg_catalog."default",
    ba_code character varying(255) COLLATE pg_catalog."default",
    province_name character varying(255) COLLATE pg_catalog."default",
    line_city_name character varying(255) COLLATE pg_catalog."default",
    line_city_level character varying(255) COLLATE pg_catalog."default",
    store_no character varying(255) COLLATE pg_catalog."default",
    terminal_name character varying(255) COLLATE pg_catalog."default",
    terminal_code character varying(255) COLLATE pg_catalog."default",
    terminal_region character varying(255) COLLATE pg_catalog."default",
    default_flag integer
)

TABLESPACE pg_default;

ALTER TABLE IF EXISTS public.new_fact_order_detail
    OWNER to postgres;

COMMENT ON COLUMN public.new_fact_order_detail.order_no
    IS '订单编号（核心主键）';

COMMENT ON COLUMN public.new_fact_order_detail.order_time
    IS '订单时间（精确到时间）';

COMMENT ON COLUMN public.new_fact_order_detail.order_date
    IS '订单日期（分析趋势或聚合）';

COMMENT ON COLUMN public.new_fact_order_detail.brand_code
    IS '品牌代码（关键分类维度）';

COMMENT ON COLUMN public.new_fact_order_detail.program_code
    IS '项目代码（分类维度）';

COMMENT ON COLUMN public.new_fact_order_detail.order_type
    IS '订单类型（正单/退单等）';

COMMENT ON COLUMN public.new_fact_order_detail.sales
    IS '销售额（核心指标）';

COMMENT ON COLUMN public.new_fact_order_detail.item_qty
    IS '商品数量（核心指标）';

COMMENT ON COLUMN public.new_fact_order_detail.item_price
    IS '单价（单项分析）';

COMMENT ON COLUMN public.new_fact_order_detail.channel
    IS '渠道（一级分类）';

COMMENT ON COLUMN public.new_fact_order_detail.subchannel
    IS '子渠道（二级分类）';

COMMENT ON COLUMN public.new_fact_order_detail.sub_subchannel
    IS '子渠道-细分（三级分类）';

COMMENT ON COLUMN public.new_fact_order_detail.material_code
    IS '产品代码（SKU分析）';

COMMENT ON COLUMN public.new_fact_order_detail.material_name_cn
    IS '产品名称（易读性）';

COMMENT ON COLUMN public.new_fact_order_detail.material_type
    IS '产品类型（分类维度）';

COMMENT ON COLUMN public.new_fact_order_detail.merged_c_code
    IS '顾客编号（客户行为分析）';

COMMENT ON COLUMN public.new_fact_order_detail.tier_code
    IS '会员等级代码（用户分层）';

COMMENT ON COLUMN public.new_fact_order_detail.first_order_date
    IS '首单日期（客户生命周期）';

COMMENT ON COLUMN public.new_fact_order_detail.is_mtd_active_member_flag
    IS 'MTD活跃客户标记（近期活跃分析）';

COMMENT ON COLUMN public.new_fact_order_detail.ytd_active_arr
    IS 'YTD活跃标记（年度活跃分析）';

COMMENT ON COLUMN public.new_fact_order_detail.r12_active_arr
    IS 'R12活跃标记（年度活跃分析）';

COMMENT ON COLUMN public.new_fact_order_detail.manager_counter_code
    IS '管理门店代码（业务归属）';

COMMENT ON COLUMN public.new_fact_order_detail.ba_code
    IS 'BA编号（关联员工表现）';

COMMENT ON COLUMN public.new_fact_order_detail.province_name
    IS '省份（地理维度）';

COMMENT ON COLUMN public.new_fact_order_detail.line_city_name
    IS '城市名称（地理维度）';

COMMENT ON COLUMN public.new_fact_order_detail.line_city_level
    IS '城市等级（地理维度）';

COMMENT ON COLUMN public.new_fact_order_detail.store_no
    IS '柜台编号（销售点）';

COMMENT ON COLUMN public.new_fact_order_detail.terminal_name
    IS '门店名称（易读性）';

COMMENT ON COLUMN public.new_fact_order_detail.terminal_code
    IS '门店代码（具体标识）';

COMMENT ON COLUMN public.new_fact_order_detail.terminal_region
    IS '区域（业务区域维度）';

COMMENT ON COLUMN public.new_fact_order_detail.default_flag
    IS '特殊订单标记（异常分析）';
""")


Adding ddl: 
CREATE TABLE IF NOT EXISTS public.new_fact_order_detail
(
    order_no character varying(255) COLLATE pg_catalog."default",
    order_time timestamp without time zone,
    order_date date,
    brand_code character varying(255) COLLATE pg_catalog."default",
    program_code character varying(255) COLLATE pg_catalog."default",
    order_type integer,
    sales numeric(18,2),
    item_qty integer,
    item_price numeric(18,2),
    channel character varying(255) COLLATE pg_catalog."default",
    subchannel character varying(255) COLLATE pg_catalog."default",
    sub_subchannel character varying(255) COLLATE pg_catalog."default",
    material_code character varying(255) COLLATE pg_catalog."default",
    material_name_cn character varying(255) COLLATE pg_catalog."default",
    material_type character varying(255) COLLATE pg_catalog."default",
    merged_c_code character varying(255) COLLATE pg_catalog."default",
    tier_code character varying(255) COLLATE pg_catalog."default",


Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Insert of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl
Add of existing embedding ID: 27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl


'27ba6359-a260-5584-b0b5-58f91fe89cc5-ddl'

In [9]:
# 添加相关的SQL训练数据以增强查询能力
vn.train(documentation="订单编号(order_no)是每个订单的唯一标识符，是表中最核心的字段。它用于追踪和唯一标识订单。")
vn.train(documentation="订单时间(order_time)记录了订单的具体时间，精确到秒，用于分析订单的时间分布和趋势。")
vn.train(documentation="订单日期(order_date)是订单的日期字段，用于根据日期进行聚合分析，发现销售和订单模式。")
vn.train(documentation="品牌代码(brand_code)用于标识商品的品牌，是分析品牌销售和市场份额的关键字段。")
vn.train(documentation="销售额(sales)字段记录了每个订单的销售总额，是衡量业务表现的核心指标。")
vn.train(documentation="商品数量(item_qty)记录了每个订单中商品的数量，是衡量销售数量和库存管理的重要数据。")
vn.train(documentation="商品单价(item_price)记录商品的单价，用于分析价格对销售的影响。")
vn.train(documentation="渠道(channel)字段表示订单的销售渠道，例如线上、电商、线下零售等，是分析不同销售渠道表现的依据。")
vn.train(documentation="子渠道(subchannel)是对渠道的细分，帮助深入分析具体的销售途径。")
vn.train(documentation="产品代码(material_code)是产品的唯一标识符，帮助分析产品的销售表现和库存情况。")
vn.train(documentation="产品名称(material_name_cn)记录产品的中文名称，提供易于阅读和理解的产品描述。")
vn.train(documentation="产品类型(material_type)用于标识产品的类别，例如电子产品、日用商品等，有助于进行产品分类分析。")
vn.train(documentation="顾客编号(merged_c_code)是每个顾客的唯一标识符，用于客户行为分析和生命周期管理。")
vn.train(documentation="会员等级代码(tier_code)用于标识顾客的会员等级，有助于进行会员分层和定向营销。")
vn.train(documentation="首单日期(first_order_date)记录了顾客的首单日期，是衡量客户生命周期和忠诚度的重要字段。")
vn.train(documentation="MTD活跃客户标记(is_mtd_active_member_flag)用于标识某一月份内活跃的客户，是客户活跃度分析的关键指标。")
vn.train(documentation="YTD活跃客户标记(ytd_active_arr)表示年度活跃客户标记，用于衡量顾客在年度内的活跃情况。")
vn.train(documentation="城市名称(line_city_name)记录了订单发生的城市名称，有助于进行地理维度的分析。")
vn.train(documentation="门店名称(terminal_name)是门店的名称，便于理解和分析不同门店的销售表现。")
vn.train(documentation="特殊订单标记(default_flag)标识是否为特殊订单，用于异常订单的分析和处理。")



Add of existing embedding ID: e51166bf-e9ec-5c3b-86d3-051efc5ac1fe-doc
Add of existing embedding ID: 109cc57b-fdc6-5bdc-b322-ad0ec25f6cb1-doc
Add of existing embedding ID: bd432e22-4daf-54d7-8b58-9d0b45dd65be-doc
Add of existing embedding ID: a90a7bc9-384e-56d8-88a5-584b72ca1a04-doc
Add of existing embedding ID: 5d236575-1044-5678-a388-d9e08ce45fb3-doc
Add of existing embedding ID: 27e9a406-a3b7-583e-8d35-b072837dd025-doc
Add of existing embedding ID: c7520444-8fc8-5f41-a87f-fe41597537b0-doc
Add of existing embedding ID: db7250b8-45e9-5918-b2e8-6a485c3f996a-doc
Add of existing embedding ID: 183bd864-de7b-5091-9a9c-bd17c91c986f-doc
Add of existing embedding ID: bd6503a8-f78a-5d47-8bf6-9d43f23f5677-doc
Add of existing embedding ID: e3d342c1-e1b6-5b51-8699-8796556fcc7d-doc
Add of existing embedding ID: 0f6fe0db-d09c-52d8-88c8-91a096d23627-doc
Add of existing embedding ID: 4be165e4-3fa0-5b7e-a27c-6f2611b09646-doc
Add of existing embedding ID: 54c69e03-ba86-5c55-9893-6508a3603698-doc
Add of

Adding documentation....
Adding documentation....
Adding documentation....


Add of existing embedding ID: 21fce156-0763-5508-81d7-b93fd106e3ec-doc
Insert of existing embedding ID: 21fce156-0763-5508-81d7-b93fd106e3ec-doc
Add of existing embedding ID: 1e77b818-79e1-5c7a-a92c-1c96cc4ffb12-doc
Insert of existing embedding ID: 1e77b818-79e1-5c7a-a92c-1c96cc4ffb12-doc
Add of existing embedding ID: d8a15a07-ddf0-53a1-8072-a0da90ac1db3-doc
Insert of existing embedding ID: d8a15a07-ddf0-53a1-8072-a0da90ac1db3-doc
Add of existing embedding ID: 2f5e03fe-8c61-5f6c-999f-91e0d30494e1-doc
Insert of existing embedding ID: 2f5e03fe-8c61-5f6c-999f-91e0d30494e1-doc
Add of existing embedding ID: 26b3078a-c6f5-5187-8fd3-2ac33a640ea7-doc
Insert of existing embedding ID: 26b3078a-c6f5-5187-8fd3-2ac33a640ea7-doc


Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


Add of existing embedding ID: 76c97a42-5e20-549a-88b7-a687288f1db6-doc
Insert of existing embedding ID: 76c97a42-5e20-549a-88b7-a687288f1db6-doc
Add of existing embedding ID: 092454fa-a261-590b-b476-c2b8be0c154d-doc
Insert of existing embedding ID: 092454fa-a261-590b-b476-c2b8be0c154d-doc
Add of existing embedding ID: c17afe4c-7936-5426-9179-1d2cc14f66e0-doc
Insert of existing embedding ID: c17afe4c-7936-5426-9179-1d2cc14f66e0-doc
Add of existing embedding ID: 0596ace1-ebda-5f25-9d89-7d8d5cfd699e-doc
Insert of existing embedding ID: 0596ace1-ebda-5f25-9d89-7d8d5cfd699e-doc


...

Add of existing embedding ID: a4887882-ea84-5637-832d-534428e5fe74-doc
Insert of existing embedding ID: a4887882-ea84-5637-832d-534428e5fe74-doc
Add of existing embedding ID: ed4fdd5d-0425-502a-bd8b-3fe43ed42410-doc
Insert of existing embedding ID: ed4fdd5d-0425-502a-bd8b-3fe43ed42410-doc
Add of existing embedding ID: 3876ec6e-4aa5-54cb-8fa1-457399dc3c8c-doc
Insert of existing embedding ID: 3876ec6e-4aa5-54cb-8fa1-457399dc3c8c-doc
Add of existing embedding ID: 758302a0-b01b-582f-a9f8-de54efcd5cec-doc
Insert of existing embedding ID: 758302a0-b01b-582f-a9f8-de54efcd5cec-doc


Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


Add of existing embedding ID: 0eeac9f2-809d-53d9-8b55-0f5241841d51-doc
Insert of existing embedding ID: 0eeac9f2-809d-53d9-8b55-0f5241841d51-doc
Add of existing embedding ID: e51166bf-e9ec-5c3b-86d3-051efc5ac1fe-doc
Insert of existing embedding ID: e51166bf-e9ec-5c3b-86d3-051efc5ac1fe-doc
Add of existing embedding ID: 109cc57b-fdc6-5bdc-b322-ad0ec25f6cb1-doc
Insert of existing embedding ID: 109cc57b-fdc6-5bdc-b322-ad0ec25f6cb1-doc
Add of existing embedding ID: bd432e22-4daf-54d7-8b58-9d0b45dd65be-doc
Insert of existing embedding ID: bd432e22-4daf-54d7-8b58-9d0b45dd65be-doc
Add of existing embedding ID: a90a7bc9-384e-56d8-88a5-584b72ca1a04-doc
Insert of existing embedding ID: a90a7bc9-384e-56d8-88a5-584b72ca1a04-doc


Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....


'a90a7bc9-384e-56d8-88a5-584b72ca1a04-doc'

In [11]:
# 按品牌查询订单数据
vn.train(sql="SELECT order_no, order_time, sales, item_qty FROM public.new_fact_order_detail WHERE brand_code = 'ABC'")

# 按订单日期范围查询销售数据
vn.train(sql="SELECT order_no, order_date, sales, channel FROM public.new_fact_order_detail WHERE order_date BETWEEN '2025-01-01' AND '2025-03-31'")

# 计算每个省份的总销售额
vn.train(sql="SELECT province_name, SUM(sales) AS total_sales FROM public.new_fact_order_detail GROUP BY province_name")

# 查询活跃会员的订单数量
vn.train(sql="SELECT merged_c_code, tier_code, COUNT(order_no) AS total_orders FROM public.new_fact_order_detail WHERE is_mtd_active_member_flag = 1 GROUP BY merged_c_code, tier_code")

# 查询特定门店的订单详细信息
vn.train(sql="SELECT order_no, item_qty, item_price, terminal_name FROM public.new_fact_order_detail WHERE store_no = 'S123'")

# 按商品代码筛选订单
vn.train(sql="SELECT order_no, material_code, material_name_cn, item_qty FROM public.new_fact_order_detail WHERE material_code = 'P123'")

# 查找销售额大于500的订单
vn.train(sql="SELECT order_no, sales, order_date, channel FROM public.new_fact_order_detail WHERE sales > 500")

# 按省份分组查询订单数量
vn.train(sql="SELECT province_name, COUNT(order_no) AS order_count FROM public.new_fact_order_detail GROUP BY province_name")

# 获取特定顾客的订单详情
vn.train(sql="SELECT order_no, item_qty, sales, order_date FROM public.new_fact_order_detail WHERE merged_c_code = 'C001'")

# 获取某个城市的订单详情
vn.train(sql="SELECT order_no, sales, channel, item_qty FROM public.new_fact_order_detail WHERE line_city_name = 'Beijing'")

# 统计每个会员等级的总销售额
vn.train(sql="SELECT tier_code, SUM(sales) AS total_sales FROM public.new_fact_order_detail GROUP BY tier_code")

# 按日期查询每个商品的销售数据
vn.train(sql="SELECT material_code, material_name_cn, sales, order_date FROM public.new_fact_order_detail WHERE order_date = '2025-03-01'")

# 查询特定渠道的销售额
vn.train(sql="SELECT channel, SUM(sales) AS total_sales FROM public.new_fact_order_detail WHERE channel = 'Online' GROUP BY channel")

# 计算每个门店的订单总数
vn.train(sql="SELECT terminal_name, COUNT(order_no) AS order_count FROM public.new_fact_order_detail GROUP BY terminal_name")

# 获取退单数据
vn.train(sql="SELECT order_no, sales, item_qty FROM public.new_fact_order_detail WHERE order_type = 2")  # 假设2代表退单类型

# 根据客户的活跃状态查询订单数据
vn.train(sql="SELECT merged_c_code, order_no, sales, item_qty FROM public.new_fact_order_detail WHERE is_mtd_active_member_flag = 1")

# 按城市等级分组查询销售总额
vn.train(sql="SELECT line_city_level, SUM(sales) AS total_sales FROM public.new_fact_order_detail GROUP BY line_city_level")

# 查询特定产品类型的订单
vn.train(sql="SELECT order_no, material_type, sales FROM public.new_fact_order_detail WHERE material_type = 'Electronics'")

# 统计每月的销售额
vn.train(sql="SELECT EXTRACT(MONTH FROM order_date) AS month, SUM(sales) AS total_sales FROM public.new_fact_order_detail GROUP BY EXTRACT(MONTH FROM order_date)")

# 查询特定城市的活跃客户数量
vn.train(sql="SELECT line_city_name, COUNT(DISTINCT merged_c_code) AS active_customers FROM public.new_fact_order_detail WHERE is_mtd_active_member_flag = 1 GROUP BY line_city_name")

Add of existing embedding ID: c81b65db-c6b0-5e6c-8b5e-52b7fbf7aa0f-sql
Insert of existing embedding ID: c81b65db-c6b0-5e6c-8b5e-52b7fbf7aa0f-sql


Question generated with sql: What are the order numbers, order times, sales, and item quantities for orders associated with the brand 'ABC'? 
Adding SQL...
Question generated with sql: What are the order numbers, dates, sales amounts, and channels for orders placed between January 1, 2025, and March 31, 2025? 
Adding SQL...
Question generated with sql: What are the total sales for each province? 
Adding SQL...
Question generated with sql: What is the total number of orders placed by active members, grouped by their merged customer code and tier code? 
Adding SQL...
Question generated with sql: What are the order numbers, item quantities, item prices, and terminal names for all orders placed at store 'S123'? 
Adding SQL...
Question generated with sql: What are the order numbers, material codes, material names in Chinese, and item quantities for the material with code 'P123'? 
Adding SQL...
Question generated with sql: What are the details of orders where the sales amount exceeded $500? 

'3da48f77-aa21-58fe-9192-40b8eb5ae672-sql'

In [12]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,f91ceec2-1b82-54b6-9fe1-c30b5048bbff-sql,What are the details of the person named John ...,SELECT * FROM my-table WHERE name = 'John Doe',sql
1,c81b65db-c6b0-5e6c-8b5e-52b7fbf7aa0f-sql,"What are the order numbers, order times, sales...","SELECT order_no, order_time, sales, item_qty F...",sql
2,94b56fd4-5fb3-545c-b41c-8c2ccdf9064e-sql,"What are the order numbers, dates, sales amoun...","SELECT order_no, order_date, sales, channel FR...",sql
3,0984926e-aa16-52e2-88d7-de8178962da3-sql,What are the total sales for each province?,"SELECT province_name, SUM(sales) AS total_sale...",sql
4,42960fa8-302d-54f3-9cda-e5748d7f101b-sql,What is the total number of orders placed by a...,"SELECT merged_c_code, tier_code, COUNT(order_n...",sql
...,...,...,...,...
232,1d8c9610-8a44-59c8-be05-9a85c9bb470c-doc,None,The following columns are in the foreign_serve...,documentation
233,b7acb8fa-5ebb-5534-a0c6-0d641e36067f-doc,None,The following columns are in the foreign_data_...,documentation
234,011e2c3a-895e-5a1f-bea3-5625ae1401ad-doc,None,The following columns are in the enabled_roles...,documentation
235,49905ef1-e2a6-5d04-982d-ec4ed9b71354-doc,None,The following columns are in the information_s...,documentation


In [ ]:
# You can remove training data if there's obsolete/incorrect information.
vn.remove_training_data(id='1-ddl')


## Asking the AI
Whenever you ask a new question, it will find the 10 most relevant pieces of training data and use it as part of the LLM prompt to generate the SQL.

In [ ]:
vn.ask(question=...)

## Launch the User Interface
![vanna-flask](https://vanna.ai/blog/img/vanna-flask.gif)

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Add of existing embedding ID: a9dbcba8-eb0f-56ff-af40-1a69f5112d11-sql
Insert of existing embedding ID: a9dbcba8-eb0f-56ff-af40-1a69f5112d11-sql
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Traceback (most recent call last):
  File "/Users/mikoo/.pyenv/versions/3.12.0/lib/python3.12/site-packages/vanna/base/base.py"

## Next Steps
Using Vanna via Jupyter notebooks is great for getting started but check out additional customizable interfaces like the
- [Streamlit app](https://github.com/vanna-ai/vanna-streamlit)
- [Flask app](https://github.com/vanna-ai/vanna-flask)
- [Slackbot](https://github.com/vanna-ai/vanna-slack)
